## 1. Estimation of $\boldsymbol{J}$

### Code 

In [1]:
# packages
using ProgressMeter, Distributions, DelimitedFiles, Random, LaTeXStrings, Plots

In [2]:
# remove randomization by setting a seed
Random.seed!(1);

In [3]:
function compute_stat(data::Matrix{Int64}, q::Int)
	m = size(data, 1)
	n = size(data, 2)
	δ = (zeros(q, q), ) .* ones(n, n)

	for i in 1:n, j in 1:n, a in 1:q, b in 1:q
		for s in 1:m
			δ[i, j][a, b] += (data[s, i] == a) * (data[s, j] == b)
		end
	end
	δ /= m

	return δ
end;

In [4]:
function maxabs_matmat(m::Matrix{Matrix{Float64}})
	max = -1
	for r in 1:size(m, 1), c in 1:size(m, 2)
		if maximum(abs.(m[r, c])) > max
			max = maximum(abs.(m[r, c]))
		end
	end

	return max
end;

In [5]:
function metropolis_hastings_step(x::Vector{Int64}, J::Matrix{Matrix{Float64}})
	n = length(x)

	# 1. draw uniformly an index and a new configuration
	k = rand(1:n)
	z = rand(1:q)
	
	# 2. compute the acceptance ratio
	a = 0
	xk_new = mod1(x[k] + z, q)
	for i in 1:n
		if i != k
			a += J[i, k][x[i], xk_new] - J[i, k][x[i], x[k]]
		end
	end
	a *= 2
	a += J[k, k][xk_new, xk_new] - J[k, k][x[k], x[k]]
	a = exp(a)
	a = min(1, a)

	# 3/4. decide whether to accept or not the new configuration
	if rand() < a
		x[k] = xk_new
	end
	
	return x
end;

In [6]:
function boltzmann_ml(δ_data::Matrix{Matrix{Float64}}, J::Matrix{Matrix{Float64}}, 
					t_burnin::Int64, t_tot::Int64, t_wait::Int64, t_max::Int64;
					λ::Float64 = 0.1, ε_max::Float64 = 1e-2, verbose::Bool = true)
	n = size(δ_data, 1)
	q = size(δ_data[1, 1], 1)
	
	x = sample(collect(1:q), n, replace = true)
	x_model = zeros(Int64, t_tot, n)
	δ_model = (zeros(q, q), ) .* ones(n, n)

	t = 0
	ε = 1
	ProgressMeter.ijulia_behavior(:clear)
	p = ProgressUnknown("learning...", spinner = true)

	while t <= t_max && ε > ε_max
		t += 1
		fill!(x_model, 0)
		x = sample(collect(1:q), n, replace = true)

		for s in 1:t_burnin
			x = metropolis_hastings_step(x, J)
		end
		for s in 1:t_tot
			for r in 1:t_wait
				x = metropolis_hastings_step(x, J)
			end
			x_model[s, :] = x
		end		
		δ_model = compute_stat(x_model, q)
		
		J = J + λ .* (δ_data - δ_model)
	
		ε = maxabs_matmat(δ_data - δ_model)

		if mod(t, t_max ÷ 10) == 0 && verbose
			ProgressMeter.next!(p; showvalues = [(:t, t), (:ε, ε)], spinner = "⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏")
		end
	end

	ProgressMeter.finish!(p)

	return δ_model, J, x_model, t, ε
end;

In [7]:
# parameters
n = 5
q = 4

# the estimation is also acceptable by imposing
# t_burnin = 250
# t_tot = 250
# t_wait = 100
# t_max = 500

t_burnin = 500
t_tot = 500
t_wait = 100
t_max = 500;

In [8]:
# δ_data
x_data = readdlm("data.dat", Int)
δ_data = compute_stat(x_data, q);

In [9]:
# boltzmann ml scheme
J = (zeros(q, q), ) .* ones(n, n)
δ_model, J, x_model, t, ε = boltzmann_ml(δ_data, J, t_burnin, t_tot, t_wait, t_max; verbose = false);

## 2. Computation of $\mathcal{F}_{i, j}$

### Code

In [10]:
# frobenius norm function
function frobenius_norm(m::Matrix{Float64})
	f = 0
	for r in 1:size(m, 1), c in 1:size(m, 2)
		f += m[r, c]^2
	end
	f = sqrt(f)
end;

In [11]:
# computation of f_{i, j} for all i, j
f = zeros(n, n)
for i in 1:n, j in 1:n
	f[i, j] = frobenius_norm(J[i, j])
end
display(f)

neighbors = [Int64[], Int64[], Int64[], Int64[], Int64[]]
for i in 1:n, j in 1:n
	if (f[i, j] > 0.2)
		append!(neighbors[i], [j])
	end
end
neighbors = mapreduce(permutedims, vcat, neighbors)
display(neighbors)

5×5 Matrix{Float64}:
 0.0218726  0.730774   0.840845   0.174598   0.110648
 0.730774   0.0177385  0.131686   0.807782   0.119461
 0.840845   0.131686   0.0259589  0.0955901  0.897827
 0.174598   0.807782   0.0955901  0.0285117  0.918476
 0.110648   0.119461   0.897827   0.918476   0.0127939

5×2 Matrix{Int64}:
 2  3
 1  4
 1  5
 2  5
 3  4

## 3. Bayesian approach

### Code

In [12]:
function boltzmann_ml_bayes(δ_data::Matrix{Matrix{Float64}}, J::Matrix{Matrix{Float64}},
					m::Int64, λ::Float64, t_burnin::Int64, t_tot::Int64, t_wait::Int64, t_max::Int64;
					μ::Float64 = 0.1, ε_max::Float64 = 1e-2, verbose::Bool = true)
	n = size(δ_data, 1)
	q = size(δ_data[1, 1], 1)

	bayes_term = (zeros(q, q), ) .* ones(n, n)
	
	x = sample(collect(1:q), n, replace = true)
	x_model = zeros(Int64, t_tot, n)
	δ_model = (zeros(q, q), ) .* ones(n, n)

	t = 0
	ε = 1
	ProgressMeter.ijulia_behavior(:clear)
	p = ProgressUnknown("learning...", spinner = true)

	while t <= t_max && ε > ε_max
		t += 1
		fill!(x_model, 0)
		x = sample(collect(1:q), n, replace = true)

		for s in 1:t_burnin
			x = metropolis_hastings_step(x, J)
		end
		for s in 1:t_tot
			for r in 1:t_wait
				x = metropolis_hastings_step(x, J)
			end
			x_model[s, :] = x
		end		
		δ_model = compute_stat(x_model, q)

		for i in 1:n, j in 1:n, a in 1:q, b in 1:q
			bayes_term[i, j][a, b] = 1 / m * λ * sign(J[i, j][a, b])
		end

		direction_bayes = δ_data - δ_model - bayes_term
		J = J + μ .* direction_bayes
	
		ε = maxabs_matmat(direction_bayes)

		if mod(t, t_max ÷ 10) == 0 && verbose
			ProgressMeter.next!(p; showvalues = [(:t, t), (:ε, ε)], spinner = "⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏")
		end
	end

	ProgressMeter.finish!(p)

	return δ_model, J, x_model, t, ε
end;

In [13]:
# boltzmann ml scheme
m = size(x_data, 1)
λ = 0.1
J_bayes = (zeros(q, q), ) .* ones(n, n)
δ_model_bayes, J_bayes, t_bayes, ε_bayes = boltzmann_ml_bayes(δ_data, J_bayes, m, λ, t_burnin, t_tot, t_wait, t_max; verbose = false);

In [14]:
# replication of point 2 with J_bayes
f_bayes = zeros(n, n)
for i in 1:n, j in 1:n
	f_bayes[i, j] = frobenius_norm(J_bayes[i, j])
end
display(f_bayes)

neighbors_bayes = [Int64[], Int64[], Int64[], Int64[], Int64[]]
for i in 1:n, j in 1:n
	if (f_bayes[i, j] > 0.2)
		append!(neighbors_bayes[i], [j])
	end
end
neighbors_bayes = mapreduce(permutedims, vcat, neighbors_bayes)
display(neighbors_bayes)

5×5 Matrix{Float64}:
 0.0187156  0.735065   0.843545   0.163826   0.124892
 0.735065   0.0174195  0.122166   0.785267   0.0903401
 0.843545   0.122166   0.0108438  0.100471   0.895399
 0.163826   0.785267   0.100471   0.0321492  0.907771
 0.124892   0.0903401  0.895399   0.907771   0.0212673

5×2 Matrix{Int64}:
 2  3
 1  4
 1  5
 2  5
 3  4